In [3]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
from model.rec_model import microgrid

In [7]:
setup = {
  "horizon": 25,
  "timestep": 1,
  "add_unbalanced_module": True,
  "loss_load_cost": 10,
  "overgeneration_cost": 2,
  "components": [
    {
      "id": "3c03f396-98f2-46c7-945c-3c95408b6355",
      "type": "LoadModule",
      "params": {
        "time_series_profile": "house"
      }
    },
    {
      "id": "32e548e7-8b69-49b6-b471-54908cd211d3",
      "type": "RenewableModule",
      "params": {
        "time_series_profile": "solar_1"
      }
    },
    {
      "id": "ba34e364-9c4a-412f-9b3c-9068335d15d3",
      "type": "BatteryModule",
      "params": {
        "min_capacity": 0,
        "max_capacity": 50,
        "max_charge": 10,
        "max_discharge": 10,
        "efficiency": 0.95,
        "battery_cost_cycle": 0,
        "init_soc": 0.5
      }
    },
    {
      "id": "ba34e364-9c4a-412f-9b3c-9068335d15d4",
      "type": "BatteryModule",
      "params": {
        "min_capacity": 0,
        "max_capacity": 150,
        "max_charge": 10,
        "max_discharge": 10,
        "efficiency": 0.95,
        "battery_cost_cycle": 0,
        "init_soc": 0.5
      }
    },
    {
      "id": "4ba2cfa4-7323-4f0a-ac7f-fa73f1177e49",
      "type": "GridModule",
      "params": {
        "max_import": 100,
        "max_export": 100,
        "cost_per_unit_co2": 0,
        "time_series_profile": "grid_1"
      }
    }
  ]
}

In [9]:
microgrid.setup(setup)

In [11]:
from pymgrid.envs import ContinuousMicrogridEnv

In [15]:
microgrid.microgrid

Microgrid([load x 1, renewable x 1, balancing x 1, battery x 2, grid x 1])

In [13]:
env = ContinuousMicrogridEnv.from_microgrid(microgrid.microgrid)

ValueError: need at least one array to concatenate